In [14]:
import torch
import torch_harmonics as th
import numpy as np
import os 
import h5py
import glob
import sys
import matplotlib.pyplot as plt
%matplotlib inline

from varsStormer import varsStormer
from stormer_utils_pangu import StormerWrapperPangu

In [2]:
def read_era5(data,vars_stormer):
    data_np = np.zeros((len(vars_stormer),128,256))
    for i,var in enumerate(vars_stormer):
        data_np[i] = data['input/{}'.format(var)][:]
    return data_np

def norm_era5(data_np,vars_stormer,stormer_wrapper):
    data_torch = torch.from_numpy(data_np)
    data_torch_norm = stormer_wrapper.inp_transform(data_torch)
    data_np_norm = data_torch_norm.numpy()
    return data_np_norm

In [3]:
forecast_dir = '/eagle/MDClimSim/mjp5595/data/stormer/stormer_forecasts_2017_norm/'
era5_dir = '/eagle/MDClimSim/tungnd/data/wb2/1.40625deg_from_full_res_1_step_6hr_h5df/train/'

vars_stormer = varsStormer().vars_stormer

stormer_wrapper = StormerWrapperPangu(
    root_dir='/eagle/MDClimSim/tungnd/data/wb2/1.40625deg_from_full_res_1_step_6hr_h5df/',
    variables=vars_stormer,
    net=None,
    base_lead_time=[6],
    ckpt=None,
    device=None,
)

checkpoint model loaded


In [10]:
init_forecasts_norm = h5py.File('/eagle/MDClimSim/mjp5595/data/stormer/stormer_init_12hr_forecast/norm_0002.h5','r')
init_forecasts_raw = h5py.File('/eagle/MDClimSim/mjp5595/data/stormer/stormer_init_12hr_forecast/raw_0002.h5','r')
analysis_forecasts_norm = h5py.File('/eagle/MDClimSim/mjp5595/data/stormer/var3d_troyB_12init_gpu0/data/forecasts_0_1201012014.h5','r')
analysis_forecasts_raw = h5py.File('/eagle/MDClimSim/mjp5595/data/stormer/var3d_troyB_12init_gpu0/data/raw_forecasts_0_1201012014.h5','r')
era5_dir = '/eagle/MDClimSim/tungnd/data/wb2/1.40625deg_from_full_res_1_step_6hr_h5df/train/'

In [13]:
# init_forecasts_raw['0'].shape # (69,128,256)
# max(init_forecasts_raw.keys()) # 96

lon = np.linspace(0,360,256)
lat = np.linspace(90,-90,128)

fig = plt.figure(figsize=(15,10))
axs = []

hrs_to_plot = [0,24,72,96]
for i,hr in enumerate(hrs_to_plot):
    gt = h5py.File(os.path.join(era5_dir,'2017_{:0>4d}.h5'.format(2+int(hr))),'r')
    init_fc = init_forecasts_raw[str(hr)]
    ana_fc = analysis_forecasts_raw[str(hr)]
    init_increment = init_fc - gt
    ana_increment = ana_fc - gt

    inc_min = min(inc_min,np.min(init_increment),np.min(ana_increment))
    inc_max = max(inc_max,np.max(init_increment),np.max(ana_increment))

perts = []
for i,hr in enumerate(hrs_to_plot):
    gt = h5py.File(os.path.join(era5_dir,'2017_{:0>4d}.h5'.format(2+int(hr))),'r')
    init_fc = init_forecasts_raw[str(hr)]
    ana_fc = analysis_forecasts_raw[str(hr)]
    
    print('Plotting hr forecast :',hr)
    init_increment = init_fc - gt
    ana_increment = ana_fc - gt


    str_int = '43' + str((i*3)+1)
    axs.append(fig.add_subplot(int(str_int)))
    perts.append(axs[-1].pcolormesh(lon, lat, init_increment[0], cmap=cm.coolwarm, norm=colors.SymLogNorm(linthresh=1,vmin=inc_min,vmax=inc_max)))

    str_int = '43' + str((i*3)+2)
    axs.append(fig.add_subplot(int(str_int)))
    perts.append(axs[-1].pcolormesh(lon, lat, ana_increment[0], cmap=cm.coolwarm, norm=colors.SymLogNorm(linthresh=1,vmin=inc_min,vmax=inc_max)))

    str_int = '43' + str((i*3)+3)
    axs.append(fig.add_subplot(int(str_int)))
    perts.append(axs[-1].pcolormesh(lon, lat, gt[0], cmap='virdis'))

'96'

In [ ]:

# Plotting the results
forecast_step = 39 
lon = np.linspace(0,360,256)
lat = np.linspace(90,-90,128)

#fig, axs = plt.subplots(3, 3, sharex = True, sharey = True, figsize = (15,10))
#fig, axs = plt.subplots(3, 3, figsize = (15,10))
fig = plt.figure(figsize=(15,10))
axs = []

# Get original forecast
axs00 = fig.add_subplot(331)
axs.append(axs00)
hf = h5py.File(os.path.join(save_dir, 'raw_forecast_noise{}.h5'.format(noise_levels[0])),'r')
print('hf.keys() :',hf.keys())
forecast_orig = hf[str(forecast_step)][0]
print('forecast_orig.shape :',forecast_orig.shape)
axs00.invert_yaxis()
pc_orig = axs00.pcolormesh(lon, lat, forecast_orig[0], cmap = 'viridis')
plt.colorbar(pc_orig, ax = axs00, label='K')
axs00.set_title('Unperturbed Forecast')

# plot perturbations
axs01 = fig.add_subplot(332, sharey=axs00, sharex=axs00)
axs.append(axs01)
perturbations = diffs[0]
cmap = colors.ListedColormap(['blue','white','red'])
bounds = [-1.5,-0.5,0.5,1.5]
#axs01.invert_yaxis()
perts = axs01.pcolormesh(lon, lat, perturbations[0], cmap=cmap)
axs01.set_title('Perturbation sign/locations')

######################################################################################################################
# This block is to get the min and max increment values
######################################################################################################################
inc_min = 0
inc_max = 0
for idx,noise_level in enumerate(noise_levels[1:]):
    print('Plotting noise level :',noise_level)
    hf = h5py.File(os.path.join(save_dir, 'raw_forecast_noise{}.h5'.format(noise_level)),'r')
    forecast_perturbed = hf[str(forecast_step)][0]

    increment = forecast_perturbed - forecast_orig
    increment = np.where(perturbations!=0,0,increment)
    inc_min = min(inc_min,np.min(increment[0]))
    inc_max = max(inc_max,np.max(increment[0]))
    #inc_min = min(inc_min,np.min(np.log(abs(increment))))
    #inc_max = max(inc_max,np.max(np.log(abs(increment))))
######################################################################################################################
######################################################################################################################

# get analysis and mse of perturbed forecasts
perts = []
noise_mses = {}
for idx,noise_level in enumerate(noise_levels[1:]):
    print('Plotting noise level :',noise_level)
    hf = h5py.File(os.path.join(save_dir, 'raw_forecast_noise{}.h5'.format(noise_level)),'r')
    forecast_perturbed = hf[str(forecast_step)][0]

    increment = forecast_perturbed - forecast_orig
    increment = np.where(perturbations!=0,0,increment)

    mse = np.mean(np.square(increment))
    noise_mses[noise_level] = mse

    str_int = '33' + str(idx+4)
    axs.append(fig.add_subplot(int(str_int), sharey=axs00, sharex=axs00))
    #axs[-1].invert_yaxis()
    #perts.append(axs[-1].pcolormesh(lon, lat, increment[0], cmap=cm.coolwarm, norm=colors.Normalize(inc_min,inc_max)))
    #perts.append(axs[-1].pcolormesh(lon, lat, increment[0], cmap='viridis', norm=colors.CenteredNorm(vmin=inc_min,vcenter=0,vmax=inc_max)))
    #perts.append(axs[-1].pcolormesh(lon, lat, increment[0], cmap=cm.coolwarm, norm=colors.CenteredNorm(vcenter=0)))

    #twoSlope = colors.TwoSlopeNorm(vmin=inc_min, vcenter=0, vmax=inc_max)
    #perts.append(axs[-1].pcolormesh(lon, lat, increment[0], cmap='viridis', norm=colors.LogNorm(inc_min,inc_max)))
    #perts.append(axs[-1].pcolormesh(lon, lat, increment[0], cmap=cm.coolwarm, norm=colors.SymLogNorm(linthresh=0.0001)))
    perts.append(axs[-1].pcolormesh(lon, lat, increment[0], cmap=cm.coolwarm, norm=colors.SymLogNorm(linthresh=1,vmin=inc_min,vmax=inc_max)))
    axs[-1].set_title('Increment w/ Perturbation={}'.format(noise_level))
    plt.colorbar(perts[-1], ax = axs[-1], label='K')

axs02 = fig.add_subplot(333)
axs.append(axs02)
#axs02._shared_y_axes.remove(axs[0, 2])
#axs02._shared_x_axes.remove(axs[0, 2])
axs02.set_yscale('log')
axs02.set_title('MSE vs Perturbation Level')
#mse_arr = np.zeros(len(noise_mses))
mse_arr = []
print('mse_arr :',mse_arr)
print('noise_mses :',noise_mses)
noise_levels_str = []
for idx,noise_level in enumerate(noise_levels[1:]):
    mse_arr.append(noise_mses[noise_level])
    noise_levels_str.append(str(noise_level))
print('mse_arr :',mse_arr)
#mse_scat = axs[0, 2].scatter(noise_levels[1:],mse_arr)
#mse_scat = axs[0, 2].bar(np.arange(len(noise_levels[1:])),noise_levels[1:],legend=mse_arr)
#mse_scat = axs[0, 2].bar(np.arange(len(noise_levels[1:])),noise_levels[1:])
mse_scat = axs02.bar(noise_levels_str,mse_arr)
#axs[0, 2].set_xlabel(noise_levels[1:])

plt.suptitle('Effects of Perturbations on 240hr Forecast')
plt.savefig(save_dir+'perturb_sameCMap_240hr.png')



In [27]:
forecasts_raw = h5py.File(os.path.join(forecast_dir,'raw_0000.h5'),'r')
forecasts_norm = h5py.File(os.path.join(forecast_dir,'norm_0000.h5'),'r')
gt_0 = h5py.File(os.path.join(era5_dir,'2017_0000.h5'),'r')
gt_6 = h5py.File(os.path.join(era5_dir,'2017_0001.h5'),'r')

In [28]:
# This Validates the forecasts in /stormer/stormer_forecasts_2017_norm/ vs ground truth

diff_init = forecasts_raw['0'][:] - read_era5(gt_0,vars_stormer)
print('diff_init :',np.max(diff_init))
diff_pred_vs_gt = forecasts_raw['6'][:] - read_era5(gt_6,vars_stormer)
print('diff_pred_vs_gt :',np.max(diff_pred_vs_gt))
diff_pred_vs_init = forecasts_raw['6'][:] - forecasts_raw['0'][:]
print('diff_pred_vs_init :',np.max(diff_pred_vs_init))

diff_init : 0.0
diff_pred_vs_gt : 305.671875
diff_pred_vs_init : 2019.2266
